In [154]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-c5u98vka
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-c5u98vka
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [155]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <iostream> // for std::ostream
#include <vector>

namespace csc485b{
namespace a2{

using node_t = int;
using edge_t = int2;

using edge_list_t = std::vector< edge_t >;
using node_list_t = std::vector< node_t >;

} // namespace a2
} // namespace csc485b


In [156]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [157]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <cassert>  // for assert()
#include <cstddef>  // std::size_t type
#include <random>   // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a2 {

/**
 * Generates and returns a vector of random edges
 * for a graph `G=(V,E)` with `n=|V|=n` and expected `m=|E|`.
 * Referred to as an Erdős-Rényi graph.
 *
 * @see https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.fast_gnp_random_graph.html#networkx.generators.random_graphs.fast_gnp_random_graph
 */
edge_list_t generate_graph( std::size_t n, std::size_t m )
{
    assert( "At most n(n-1) edges in a simple graph" && m < n * ( n - 1 ) );

    int const probability = ( 100 * m ) / ( n * ( n - 1 ) );

    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20241008;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 100);

    edge_list_t random_edges;
    random_edges.reserve( 2 * m );

    for( node_t u = 0; u < n; ++u )
    {
        for( node_t v = u + 1; v < n; ++v )
        {
            auto const dice_roll = distrib( rng );
            if( dice_roll <= probability )
            {
                random_edges.push_back( make_int2( u, v ) );
                random_edges.push_back( make_int2( v, u ) );
            }
        }
    }
    random_edges.resize( random_edges.size() );


    return random_edges;
}

} // namespace a2
} // namespace csc485b


In [158]:
%%cuda_group_save -g "source" -n "dense_graph.h"
/**
 * The file in which you will implement your DenseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {

/**
 * A DenseGraph is optimised for a graph in which the number of edges
 * is close to n(n-1). It is represented using an adjacency matrix.
 */
struct DenseGraph
{
  std::size_t n; /**< Number of nodes in the graph. */
  node_t * adjacencyMatrix; /** Pointer to an n x n adj. matrix */

  /** Returns number of cells in the adjacency matrix. */
  __device__ __host__ __forceinline__
  std::size_t matrix_size() const { return n * n; }
};


namespace gpu {

/**
 * Constructs a DenseGraph from an input edge list of m edges.
 *
 * @pre The pointers in DenseGraph g have already been allocated.
 */
__global__
void add_neighbour_pair( DenseGraph *g, edge_t const * edge_list, std::size_t m)
{
    /*
      Consideration: If we are worried about m > avail threads on GPU, then
      we can have each thread be responsible for > 1 edge.
    */

    const std::size_t th_idx = blockIdx.x * blockDim.x + threadIdx.x;
    // Results in index access pattern of 0, 3, 4, 7, 8, ... to facilitate coalescing
    const std::size_t edge_idx = ( th_idx * 2 ) + ( th_idx % 2 );

    if ( edge_idx < m ){

        const int2 edge = edge_list[edge_idx];
        g->adjacencyMatrix[(edge.x * g->n) + edge.y] = 1;
        g->adjacencyMatrix[(edge.y * g->n) + edge.x] = 1;

    }

    return;
}

void build_graph( DenseGraph *g, edge_t const * edge_list, std::size_t m, std::size_t n )
{

    std::size_t const threads_per_block = 1024;
    std::size_t const num_blocks =  ( n + threads_per_block - 1 ) / threads_per_block;

    add_neighbour_pair<<< num_blocks, threads_per_block>>>(g, edge_list, m);
    return;
}


/**
  * Repopulates the adjacency matrix as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( DenseGraph *g )
{
    // IMPLEMENT ME!
    // square adjacencyMatrix
    // then remove the diagonal and clamp values back to [0,1]
    return;
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [159]:
%%cuda_group_save -g "source" -n "sparse_graph.h"
/**
 * The file in which you will implement your SparseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {

/**
 * A SparseGraph is optimised for a graph in which the number of edges
 * is close to cn, for a small constanct c. It is represented in CSR format.
 */
struct SparseGraph
{
  std::size_t n; /**< Number of nodes in the graph. */
  std::size_t m; /**< Number of edges in the graph. */
  node_t * neighbours_start_at; /** Pointer to an n=|V| offset array */
  node_t * neighbours; /** Pointer to an m=|E| array of edge destinations */
};


namespace gpu {


__global__
void histogram( edge_t const *arr, std::size_t m, SparseGraph *g)
{
    const std::size_t global_th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if (global_th_id < g->m)
    {
      const node_t incident_vertex = arr[global_th_id].x;
      if (incident_vertex < g->n)
      {
        //g->neighbours_start_at[incident_vertex + 1] = 1;
        atomicAdd(g->neighbours_start_at + incident_vertex + 1, 1);
      }
    }

    return;
}

/* IN-PLACE prefix sum
 * TODO: move to header file
 */

template<typename T>
__global__
void prefix_sum( DenseGraph *g, T *block_sums, std::size_t n )
{
    const std::size_t global_th_idx = blockIdx.x * blockDim.x + threadIdx.x;
    const std::size_t th_idx = threadIdx.x;

    __shared__ T smem[1024];
    smem[th_idx] = g->adjacencyMatrix[global_th_idx % n];
    __syncthreads();

    for (std::size_t stride = 1; stride < 1024; stride <<= 1)
    {
        std::size_t val = 0;

        if ( th_idx >= stride ){
            val = smem[th_idx - stride];
        }

        /*
          Needs to be a sync here.
          A fench only guarantees a strong ordering and flushing. But it does not guarantee
          that all threads will see the flushed value before it is accessed.
        */
        __syncthreads(); // Maybe can use a fence ?? -- Confirm with Sean

        if ( th_idx >= stride ){

            smem[th_idx] +=  val;
        }
         __syncthreads();

    }

    if (global_th_idx < n ){
        g->adjacencyMatrix[global_th_idx] = smem[th_idx];
    }
    __syncthreads();

    // Since we only launch a 1D kernel, the blockIdx.x's are unique
    if ( global_th_idx < n && (global_th_idx % 1024 == 1023 || global_th_idx == n -1 )  ){
        block_sums[blockIdx.x] = smem[th_idx];
    }


    return;
}


template<typename T>
__global__
void prefix_sum( SparseGraph *g, T *block_sums, std::size_t n )
{
    const std::size_t global_th_idx = blockIdx.x * blockDim.x + threadIdx.x;
    const std::size_t th_idx = threadIdx.x;

    __shared__ T smem[1024];
    smem[th_idx] = g->neighbours_start_at[global_th_idx % n];
    __syncthreads();

    for (std::size_t stride = 1; stride < 1024; stride <<= 1)
    {
        std::size_t val = 0;

        if ( th_idx >= stride ){
            val = smem[th_idx - stride];
        }

        /*
          Needs to be a sync here.
          A fench only guarantees a strong ordering and flushing. But it does not guarantee
          that all threads will see the flushed value before it is accessed.
        */
        __syncthreads(); // Maybe can use a fence ?? -- Confirm with Sean

        if ( th_idx >= stride ){

            smem[th_idx] +=  val;
        }
         __syncthreads();

    }

    if (global_th_idx < n ){
        g->neighbours_start_at[global_th_idx] = smem[th_idx];
    }
    __syncthreads();

    // Since we only launch a 1D kernel, the blockIdx.x's are unique
    if ( global_th_idx < n && (global_th_idx % 1024 == 1023 || global_th_idx == n -1 )  ){
        block_sums[blockIdx.x] = smem[th_idx];
    }


    return;
}

template<typename T>
__global__
void prefix_sum( T *arr, std::size_t n )
{
    const std::size_t global_th_idx = blockIdx.x * blockDim.x + threadIdx.x;
    const std::size_t th_idx = threadIdx.x;

    __shared__ T smem[1024];
    smem[th_idx] = arr[global_th_idx % n];
    __syncthreads();

    for (std::size_t stride = 1; stride < 1024; stride <<= 1)
    {
        std::size_t val = 0;

        if ( th_idx >= stride ){
            val = smem[th_idx - stride];
        }

        /*
          Needs to be a sync here.
          A fench only guarantees a strong ordering and flushing. But it does not guarantee
          that all threads will see the flushed value before it is accessed.
        */
        __syncthreads(); // FENCE

        if ( th_idx >= stride ){

            smem[th_idx] +=  val;
        }
         __syncthreads();

    }

    if (global_th_idx < n ){
        arr[global_th_idx] = smem[th_idx];
    }

    return;
}


template<typename T>
__global__
void prefix_sum_naive( T *arr, std::size_t n, std::size_t stride)
{
    const std::size_t th_idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (th_idx < n && th_idx >= stride){
        arr[th_idx] +=  arr[th_idx - stride];
    }

    return;
}

template<typename T>
 __global__
void finish_prefix_sum(DenseGraph *g, T* block_sums, std::size_t n)
{
    const std::size_t th_idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (th_idx < n && blockIdx.x > 0){
        g->adjacencyMatrix[th_idx] += block_sums[blockIdx.x -1 ];
    }

    return;
}


template<typename T>
 __global__
void finish_prefix_sum(T *arr, T* block_sums, std::size_t n)
{
    const std::size_t th_idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (th_idx < n && blockIdx.x > 0){
        arr[th_idx] += block_sums[blockIdx.x -1 ];
    }

    return;
}


__global__
void bucket_sort( SparseGraph *g, const edge_t *edges )
{
    const std::size_t global_th_id = blockIdx.x * blockDim.x + threadIdx.x;


    if (global_th_id < g->m)
    {
        edge_t edge = edges[global_th_id];
        node_t this_vertex  = edge.x;
        node_t other_vertex = edge.y;

        int pos = atomicAdd(g->neighbours_start_at + this_vertex, 1);
        g->neighbours[pos] = other_vertex;
    }

    return;
}


/**
 * Constructs a SparseGraph from an input edge list of m edges.
 *
 * @pre The pointers in SparseGraph g have already been allocated.
 */
//__global__
void build_graph( SparseGraph *g, edge_t const * edge_list, std::size_t m, std::size_t n )
{
    std::size_t const threads_per_block = 1024;
    std::size_t const num_blocks =  ( n + threads_per_block - 1 ) / threads_per_block;

    node_t *tmp_blk_sums, *temp_prefix_sums;
    cudaMalloc( (void**) &tmp_blk_sums, sizeof(node_t) * num_blocks );

    histogram<<< num_blocks, threads_per_block >>>( edge_list, m, g );


    // prefix_sum
    prefix_sum<<< num_blocks, threads_per_block >>>( g, tmp_blk_sums, n+1 );
    //prefix_sum<<< 1 , num_blocks >>>( tmp_blk_sums, inter.size(), false );
    //finish_prefix_sum<<< num_blocks, threads_per_block >>>( tmp_blk_sums, tmp_blk_sums, input.size() );

    cudaFree(tmp_blk_sums);


    // bucket_sort
    bucket_sort<<< num_blocks, threads_per_block >>>(  g, edge_list );

    return;
}

/**
  * Repopulates the adjacency lists as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( SparseGraph *g )
{
    // IMPLEMENT ME!
    // algorithm unknown
    return;
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [160]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <chrono>   // for timing
#include <iostream> // std::cout, std::endl
#include <iterator> // std::ostream_iterator
#include <vector>

#include "dense_graph.h"
#include "sparse_graph.h"

#include "data_generator.h"
#include "data_types.h"

/**
 * Runs timing tests on a CUDA graph implementation.
 * Consists of independently constructing the graph and then
 * modifying it to its two-hop neighbourhood.
 */
template < typename DeviceGraph >
void run( DeviceGraph *g, csc485b::a2::edge_t const * d_edges, std::size_t m, std::size_t n )
{
    cudaDeviceSynchronize();
    auto const build_start = std::chrono::high_resolution_clock::now();

    // this code doesn't work yet!
    //csc485b::a2::gpu::build_graph<<< 1, 1 >>>( g, d_edges, m );
    csc485b::a2::gpu::build_graph( g, d_edges, m, n );

    cudaDeviceSynchronize();
    auto const reachability_start = std::chrono::high_resolution_clock::now();

    // neither does this!
    csc485b::a2::gpu::two_hop_reachability<<< 1, 1 >>>( g );

    cudaDeviceSynchronize();
    auto const end = std::chrono::high_resolution_clock::now();

    std::cout << "Build time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(reachability_start - build_start).count()
              << " us"
              << std::endl;

    std::cout << "Reachability time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(end - reachability_start).count()
              << " us"
              << std::endl;
}

/**
 * Allocates space for a dense graph and then runs the test code on it.
 */
void run_dense( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device DenseGraph
    a2::node_t * d_matrix;
    cudaMalloc( (void**)&d_matrix, sizeof( a2::node_t ) * n * n );
    a2::DenseGraph dg{ n, d_matrix };
    a2::DenseGraph *d_dg;
    cudaMalloc( (void**)&d_dg, sizeof( a2::SparseGraph ) );
    cudaMemcpy( d_dg, &dg, sizeof( a2::SparseGraph ), cudaMemcpyHostToDevice );

    run( d_dg, d_edges, m, n );

    // check output?
    std::vector< a2::node_t > host_matrix( dg.matrix_size() );
    a2::DenseGraph dg_res{ n, host_matrix.data() };
    cudaMemcpy( dg_res.adjacencyMatrix, dg.adjacencyMatrix, sizeof( a2::node_t ) * dg.matrix_size(), cudaMemcpyDeviceToHost );
    //std::copy( host_matrix.cbegin(), host_matrix.cend(), std::ostream_iterator< a2::node_t >( std::cout, " " ) );
    for (int idx = 0; idx < n; idx++)
    {
        for (int jdx = 0; jdx < n; jdx++)
        {
            std::cout << dg_res.adjacencyMatrix[idx*n + jdx] << " ";
        }
        std::cout << "\n";
    }

    // clean up
    cudaFree( d_matrix );
}

/**
 * Allocates space for a sparse graph and then runs the test code on it.
 */
void run_sparse( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device SparseGraph
    a2::node_t * d_offsets, * d_neighbours;
    cudaMalloc( (void**)&d_offsets,    sizeof( a2::node_t ) * (n+1) );
    cudaMalloc( (void**)&d_neighbours, sizeof( a2::node_t ) * m );
    a2::SparseGraph sg{n, m, d_offsets, d_neighbours };
    a2::SparseGraph *d_sg;
    cudaMalloc( (void**)&d_sg, sizeof( a2::SparseGraph ) );
    cudaMemcpy( d_sg, &sg, sizeof( a2::SparseGraph ), cudaMemcpyHostToDevice );
    //a2::SparseGraph d_sg{ n, m, d_offsets, d_neighbours };

    run( d_sg, d_edges, m, n );

    // check output
    a2::SparseGraph *sg_res;
    a2::node_t *offsets, *neighbours;
    sg_res = (a2::SparseGraph*)malloc(sizeof( a2::SparseGraph));
    offsets = (a2::node_t*)malloc(sizeof( a2::node_t) * n);
    neighbours = (a2::node_t*)malloc(sizeof( a2::node_t) * m);
    cudaMemcpy( sg_res, d_sg, sizeof( a2::SparseGraph ), cudaMemcpyDeviceToHost );
    cudaMemcpy( offsets, sg_res->neighbours_start_at, sizeof( a2::node_t ) * (n+1), cudaMemcpyDeviceToHost );
    cudaMemcpy( neighbours, d_neighbours, sizeof( a2::node_t ) * m, cudaMemcpyDeviceToHost );

    std::cout << "m: " << sg_res->m << " n: " << sg_res->n << "\n";

    for (int idx = 0; idx < n+1; idx++)
    {
        std::cout << offsets[idx] << " ";
    }
    std::cout << "\n";

    for (int idx = 0; idx < m; idx++)
    {
        std::cout << neighbours[idx] << " ";
    }
    std::cout << "\n";

    // clean up
    cudaFree( d_neighbours );
    cudaFree( d_offsets );
    cudaFree( d_sg );
    free(offsets);
    free(neighbours);
}

int main()
{
    using namespace csc485b;

    // Create input
    std::size_t constexpr n = 4;
    std::size_t constexpr expected_degree = n >> 2;

    a2::edge_list_t const graph = a2::generate_graph( n, n * expected_degree );
    std::size_t const m = graph.size();

    // lazily echo out input graph
    for( auto const& e : graph )
    {
        std::cout << "(" << e.x << "," << e.y << ") ";
    }

    // allocate and memcpy input to device
    a2::edge_t * d_edges;
    cudaMalloc( (void**)&d_edges, sizeof( a2::edge_t ) * m );
    cudaMemcpyAsync( d_edges, graph.data(), sizeof( a2::edge_t ) * m, cudaMemcpyHostToDevice );

    // run your code!
    run_sparse( d_edges, n, m );
    run_dense ( d_edges, n, m );

    return EXIT_SUCCESS;
}

In [161]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

(0,2) (2,0) (0,3) (3,0) (1,2) (2,1) (1,3) (3,1) (2,3) (3,2) Build time: 225 us
Reachability time: 15 us
m: 10 n: 4
2 4 7 10 10 
2 3 2 3 0 1 3 0 1 2 
Build time: 20 us
Reachability time: 19 us
2 4 1 1 
10 0 1 1 
1 1 0 1 
1 1 1 0 

